In [1]:
import pandas as pd
import numpy as np

In [2]:
loser = pd.read_csv('loser.csv')
winner = pd.read_csv('winner.csv')

#print(loser.head())
print(len(loser.index))
#print(winner.head())
print(len(winner.index))

print(winner.columns)

15531
15531
Index(['unique_round_id', 'ending_round', 'ending_time', 'fighter_name',
       'round_number', 'knockdowns', 'submission_attempts', 'takedowns',
       'takedowns_attempted', 'significant_strikes_head_landed',
       'significant_strikes_head_attempted', 'significant_strikes_body_landed',
       'significant_strikes_body_attempted', 'significant_strikes_leg_landed',
       'significant_strikes_leg_attempted', 'fighter_control_time',
       'opponent_control_time'],
      dtype='object')


In [3]:
df = pd.concat([winner,loser],axis=0)
df.to_csv('output.csv',index=False)
#print(df.head())
print(len(df.index))

31062


In [4]:
#fixing control time
df.fighter_control_time.loc[df['fighter_control_time']=='--']='0:00'
df.fighter_control_time=df.fighter_control_time.fillna('6:00')
df['seconds']=df.fighter_control_time.str.split(':',expand=True)[1]
df['minute']=df.fighter_control_time.str.split(':',expand=True)[0]

#changing data types
df['minute']=df.minute.astype(np.int64)
df['seconds']=df.seconds.astype(np.int64)
df['fighter_control_time']=df.seconds+(df.minute*60)

#dropping extra
df=df.drop(columns=['seconds','minute'])

c:\users\jason\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
#fixing control time
df.opponent_control_time.loc[df['opponent_control_time']=='--']='0:00'
df.opponent_control_time=df.opponent_control_time.fillna('6:00')
df['seconds']=df.opponent_control_time.str.split(':',expand=True)[1]
df['minute']=df.opponent_control_time.str.split(':',expand=True)[0]

#changing data types
df['minute']=df.minute.astype(np.int64)
df['seconds']=df.seconds.astype(np.int64)
df['opponent_control_time']=df.seconds+(df.minute*60)

#dropping extra
df=df.drop(columns=['seconds','minute'])

In [6]:
#fixing control time
df.ending_time.loc[df['ending_time']=='--']='0:00'
df.ending_time=df.ending_time.fillna('6:00')
df['seconds']=df.ending_time.str.split(':',expand=True)[1]
df['minute']=df.ending_time.str.split(':',expand=True)[0]

#changing data types
df['minute']=df.minute.astype(np.int64)
df['seconds']=df.seconds.astype(np.int64)
df['ending_time']=df.seconds+(df.minute*60)

#dropping extra
df=df.drop(columns=['seconds','minute'])

In [7]:
#there are 6308 missing values that I replaced with 6:00 since it is impossible may need to filter on this in the future because it will mess up reslts

#print(df.fighter_control_time.value_counts())
#print(df.ending_time.value_counts())
#print(df.opponent_control_time.value_counts())

#pulling out 360 since they are nulls
df=df.loc[df['fighter_control_time']!=360]
df=df.loc[df['ending_time']!=360]
df=df.loc[df['opponent_control_time']!=360]
print(len(df.index))

#print(df.fighter_control_time.value_counts())
#print(df.ending_time.value_counts())
#print(df.opponent_control_time.value_counts())

22472


In [8]:
#print(df.columns)

###need to create standing time column

#separating frames
full=df.loc[df['ending_round']!=df['round_number']]
partial=df.loc[df['ending_round']==df['round_number']]

In [9]:
#doing the math
full['standing_time']=300-full['fighter_control_time']-full['opponent_control_time']
partial['standing_time']=partial['ending_time']-partial['fighter_control_time']-partial['opponent_control_time']

c:\users\jason\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\jason\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
#reforming data frame
df_fixed = pd.concat([full,partial],axis=0)

print(len(df_fixed.index))

22472


In [11]:
#creating total round time column
df_fixed['total_round_time']=df_fixed['standing_time']+df_fixed['fighter_control_time']+df_fixed['opponent_control_time']
df_fixed=df_fixed.loc[df_fixed['total_round_time']>=150]

time = df_fixed[['total_round_time','unique_round_id']]

print(len(df_fixed.index))

20314


In [12]:
#grappling time
df_fixed['grappling_time']=df_fixed['fighter_control_time']+df_fixed['opponent_control_time']

In [13]:
#% control vs controlled
df_fixed['%_grappling_control']=df_fixed['fighter_control_time']/df_fixed['total_round_time']
df_fixed['%_grappling_controlled']=df_fixed['opponent_control_time']/df_fixed['total_round_time']

In [14]:
#ratio of time spent
df_fixed['%_time_standing']=df_fixed['standing_time']/df_fixed['total_round_time']
df_fixed['%_time_grappling']=df_fixed['grappling_time']/df_fixed['total_round_time']

In [15]:
#ratio of strikes by time
#df_fixed['standing_attempts_per_standing']=df_fixed['significant_strikes_standing_attempted']/df_fixed['total_round_time']
#df_fixed['grappling_attempts_per_grappling']=(df_fixed['significant_strikes_clinch_attempted']+df_fixed['significant_strikes_ground_attempted'])/df_fixed['total_round_time']

df_fixed=df_fixed.replace(np.inf,0)
df_fixed=df_fixed.fillna(0)

#strike mix
#df_fixed['standing_strikes_mix']=df_fixed['standing_attempts_per_standing']/(df_fixed['standing_attempts_per_standing']+df_fixed['grappling_attempts_per_grappling'])
#df_fixed['grappling_strikes_mix']=df_fixed['grappling_attempts_per_grappling']/(df_fixed['standing_attempts_per_standing']+df_fixed['grappling_attempts_per_grappling'])
#df_fixed['standing_strikes_mix']=df_fixed['significant_strikes_standing_attempted']/((df_fixed['significant_strikes_clinch_attempted']+df_fixed['significant_strikes_ground_attempted'])+df_fixed['significant_strikes_standing_attempted'])
#df_fixed['grappling_strikes_mix']=(df_fixed['significant_strikes_clinch_attempted']+df_fixed['significant_strikes_ground_attempted'])/(df_fixed['significant_strikes_standing_attempted']+(df_fixed['significant_strikes_clinch_attempted']+df_fixed['significant_strikes_ground_attempted']))

In [16]:
print(df_fixed.columns)

Index(['unique_round_id', 'ending_round', 'ending_time', 'fighter_name',
       'round_number', 'knockdowns', 'submission_attempts', 'takedowns',
       'takedowns_attempted', 'significant_strikes_head_landed',
       'significant_strikes_head_attempted', 'significant_strikes_body_landed',
       'significant_strikes_body_attempted', 'significant_strikes_leg_landed',
       'significant_strikes_leg_attempted', 'fighter_control_time',
       'opponent_control_time', 'standing_time', 'total_round_time',
       'grappling_time', '%_grappling_control', '%_grappling_controlled',
       '%_time_standing', '%_time_grappling'],
      dtype='object')


In [17]:
#some data comes out null and with infinites, replacing with 0
df_fixed=df_fixed.fillna(0)
df_fixed=df_fixed.replace(np.inf,0)

df_fixed=df_fixed[['unique_round_id','%_grappling_control','%_time_standing']]

df_fixed.to_csv('Modeling/modeling.csv',index=False)
print(df_fixed.head())

   unique_round_id  %_grappling_control  %_time_standing
0           1350.0             0.003333         0.996667
2            248.0             0.293333         0.660000
3            250.0             0.203333         0.763333
5          10434.0             0.483333         0.500000
7           7356.0             0.000000         1.000000


In [18]:
comparer=pd.read_csv('winner_comparer.csv')
comparer=comparer.merge(time, on='unique_round_id',how='inner')
comparer=comparer[['unique_round_id','total_round_time','knockdowns', 'total_strikes_landed',
       'total_strikes_attempted', 'takedowns', 'takedowns_attempted',
       'submission_attempts', 'control_time', 'reversals', 'guard_passes',
       'significant_strikes_head_landed', 'significant_strikes_head_attempted',
       'significant_strikes_body_landed', 'significant_strikes_body_attempted',
       'significant_strikes_leg_landed', 'significant_strikes_leg_attempted',
       'significant_strikes_standing_landed',
       'significant_strikes_standing_attempted',
       'significant_strikes_clinch_landed',
       'significant_strikes_clinch_attempted',
       'significant_strikes_ground_landed',
       'significant_strikes_ground_attempted']]

comparer.to_csv('Modeling/comparer.csv',index=False)